In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

import dataset

Data

In [2]:
IMG_HEIGHT = 192
IMG_WIDTH = 192
BATCH_SIZE = 32
EPOCHS = 3

In [3]:
labels = ["hongniu", "wangqiu", "xuehua"]
data_path_label = dataset.load_image_path(labels)

In [4]:
import random
data_path_label = list(data_path_label.items())
random.shuffle(data_path_label)
data_path_label[:10]

[('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/xuehua/img00239.jpg',
  2),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/wangqiu/img00426.jpg',
  1),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/wangqiu/img00474.jpg',
  1),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/hongniu/img00037.jpg',
  0),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/xuehua/img00286.jpg',
  2),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/wangqiu/img00254.jpg',
  1),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/xuehua/img00599.jpg',
  2),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/xuehua/img00469.jpg',
  2),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/xuehua/img00399.jpg',
  2),
 ('/Users/land/Documents/Workspace/ML/tf/OmegaLab/dataset/wangqiu/img00165.jpg',
  1)]

In [5]:
def load_image(path):
    image_raw = tf.io.read_file(path)
    image = tf.image.decode_image(image_raw, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image /= 255.0
    return image

In [6]:
num_data = len(data_path_label)
num_train = int(num_data * 0.8)
print(num_data, num_train)

1103 882


In [7]:
train_path_label = data_path_label[:num_train]
val_path_label = data_path_label[num_train:]

In [8]:
train_paths, train_labels = zip(*train_path_label)
val_paths, val_labels = zip(*val_path_label)

In [9]:
train_images = list(map(load_image, train_paths))
val_images = list(map(load_image, val_paths))
train_labels = list(train_labels)
val_labels = list(val_labels)

In [10]:
train_image_label_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_image_label_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

In [11]:
train_batches = train_image_label_ds.shuffle(num_train).repeat().batch(BATCH_SIZE)
val_batches = val_image_label_ds.shuffle(num_data - num_train).repeat().batch(BATCH_SIZE)

In [12]:
def create_model():
    # vgg16 = keras.applications.VGG16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights='imagenet')
    # vgg16.trainable = False
    model = keras.Sequential([
        keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        keras.layers.MaxPooling2D(),
        keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        keras.layers.MaxPooling2D(),
        keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        keras.layers.MaxPooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ])
        # keras.Sequential([
    #     vgg16,
    #     keras.layers.GlobalAveragePooling2D(),
    #     keras.layers.Dense(len(labels), activation='softmax')
    # ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
    # return

In [13]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 192, 192, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 96, 96, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0

In [14]:
model_log = model.fit(train_batches,
                      epochs=EPOCHS,
                      steps_per_epoch=num_train,
                      validation_data=val_batches,
                      validation_steps=num_data-num_train)

Train for 882 steps, validate for 221 steps
Epoch 1/3
882/882 [==============================] - 488s 553ms/step - loss: 0.0597 - accuracy: 0.9853 - val_loss: 0.0024 - val_accuracy: 1.0000

In [15]:
model.save('cnn.h5')